# Two Newsvendor Problem

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pedronahum/stochastic-optimization/blob/master/notebooks/two_newsvendor.ipynb)

In [ ]:
# Install JAX and dependencies
!pip install -q jax jaxlib jaxtyping chex numpy matplotlib

# Clone repository (force fresh clone for latest code)
import os
import shutil

if os.path.exists('stochastic-optimization'):
    shutil.rmtree('stochastic-optimization')

!git clone https://github.com/pedronahum/stochastic-optimization.git
os.chdir('stochastic-optimization')

# Clear Python import cache
import sys
for key in list(sys.modules.keys()):
    if key.startswith('problems'):
        del sys.modules[key]

print('✓ Setup complete!')

In [ ]:
# Import the two newsvendor componentsfrom problems.two_newsvendor import (    TwoNewsvendorConfig,    TwoNewsvendorFieldModel,    NewsvendorFieldPolicy)import jaximport jax.numpy as jnp# Create newsvendor policypolicy = NewsvendorFieldPolicy(model)print('✓ Model and policy ready')

In [ ]:
# Create model configuration
config = TwoNewsvendorConfig(
    demand_lower=0.0,
    demand_upper=100.0,
    overage_cost_field=1.0,
    underage_cost_field=9.0
)
model = TwoNewsvendorFieldModel(config)
key = jax.random.PRNGKey(42)
state = model.init_state(key)
print('✓ Two newsvendor model ready')

In [ ]:
# Simulate coordination
orders, allocations, rewards = [], [], []
for t in range(30):
    key, k1, k2 = jax.random.split(key, 3)
    decision = policy(None, state, k1, model)
    exog = model.sample_exogenous(k2, state, t)
    reward = model.reward(state, decision, exog)
    orders.append(decision[:2].tolist())
    allocations.append(decision[2] if len(decision) > 2 else 0)
    rewards.append(float(reward))
    state = model.transition(state, decision, exog)

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot([o[0] for o in orders], label='Newsvendor 1')
plt.plot([o[1] for o in orders], label='Newsvendor 2')
plt.title('Order Quantities')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(np.cumsum(rewards))
plt.title('Cumulative Profit')
plt.tight_layout()
plt.show()